In [2]:
from bs4 import BeautifulSoup as bs
import requests
import datetime, time
import pickle

import pandas as pd
import numpy as np

import os
import sys
import io
import shutil

import matplotlib.pyplot as plt

In [3]:
# !pip install cfscrape # 403 forbidden, cloudflare error을 해결하기 위한 모듈
import cloudscraper
scraper = cloudscraper.create_scraper()

In [4]:
# # !pip install cfscrape # 403 forbidden, cloudflare error을 해결하기 위한 모듈
# import cfscrape
# scraper = cfscrape.create_scraper()
# # 이후 403 error이 발생한 곳에는 requests 대신 scraper 사용

In [5]:
headers = {'Referer': 'https://kr.investing.com/',
           'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
           AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36 Edg/125.0.0.0'}

In [6]:
def concat_df(df_o, df, dup_col, sort_col):
    df_o = pd.concat([df_o, df], ignore_index=True)
    df_o.drop_duplicates(subset=[dup_col], keep='last', inplace=True) # dup_col 중첩제거 기준 컬럼 이름: "time", "date" 등
#     df_o.drop_duplicates(subset=[dup_col], keep='first', inplace=True)
    df_o.sort_values(by=[df_o.columns[sort_col]], inplace=True) # sort_col 정렬 기준 컬럼 번호
    df_o.index = np.arange(0, len(df_o))  # 일련 번호 오름차순으로 재 설정
    return df_o

## 시간별 시세

In [7]:
# 특정일의 시간별 시세 취득을 위한 페이지별 table 데이터 취득. page는 날짜별로 약 40쪽. 09:00:00부터 1분간격으로 15:58:00 까지.
def get_piece_time_price(url_t):
    res = scraper.get(url_t, headers=headers)
    class_name = 'type2'
    df = pd.read_html(io.StringIO(str(res.text)), attrs={"class": class_name}, flavor=["lxml", "bs4"])[0]
    
    df = df.dropna(axis=0) # delete rows with nan values
    df.columns = ['time', 'price', 'change', 'sell', 'buy', 'volume', 'change_volumn'] # rename column in english from korean

    # convert character values to integer value : 보합= 0, 하락= -, 상승= +
    df['change'] = df['change'].apply(lambda x: int(x[2:]) if x[:2] == '보합' 
                                      else (-int(x[4:].replace(',','')) if x[:2] == '하락' 
                                            else int(x[4:].replace(',',''))))  # convert characters to int
    
    df = df[['time', 'price', 'change', 'volume']]  # delete unnecessary columns

    # define variable types
    df['time'] = df['time'].apply(lambda x : datetime.datetime.strptime(x, "%H:%M").time())  # convert characters to datetime objet
    df['price'] = df['price'].astype(int)
    df['volume'] = df['volume'].astype(int)

    return df

In [8]:
# 특정일의 시간대별 가격 40쪽을 한개의 df로 묶는 기능
def get_time_price(url_base_t, code_com, collect_date):
    
    page_num = 1
    
    # make first data frame
    collect_date_time = collect_date + '160000' # 장종료후 16시 00분 00초에 시간별 시세 추출
    page = str(page_num)
    
    url = url_base_t + '?code=' + code_com + '&thistime=' + collect_date_time + '&page=' + page
    df_base = get_piece_time_price(url)
    
    page_num = page_num + 1
    
    while True:
        page = str(page_num)
        
        url = url_base_t + '?code=' + code_com + '&thistime=' + collect_date_time + '&page=' + page
        df_p = get_piece_time_price(url)
    
        df_base = concat_df(df_base, df_p, 'time', 0)  # df concat후 'time' column을 기준으로 중복제거 후 0 column을 기준으로 정렬시킴.
        # print('page_num', page_num)
        
        if df_p['time'].iloc[-1] == datetime.time(9, 00):
            break
    
        page_num = page_num + 1
    
    df_base['date'] = datetime.datetime.strptime(collect_date, '%Y%m%d') # insert column with collecting date
    df_base = df_base[['date', 'time', 'price', 'change', 'volume']]  

    return df_base

In [9]:
# naver_dir = 'data/naver_finance/time_data'
naver_dir = 'time_data'

url_base = 'https://finance.naver.com/item/sise_time.naver'  # sise_date

code_dic = {'005930': ['삼성전자', 'sec'], '005380': ['현대차', 'hyunmotor'], 
        '035420': ['NAVER', 'naver'], '033780': ['KT&G', 'ktng']}
# code_list = list(code_dic.items())
# code_company_name = code_list[0]
# code = code_company_name[0] # 취득을 원하는 회사 주식 코드
code_dic = {'005930': ['삼성전자', 'sec'], '005380': ['현대차', 'hyunmotor'], }
code_dic = {'035420': ['NAVER', 'naver'], '033780': ['KT&G', 'ktng']}

c_date = ['20240619', '20240620', '20240621', '20240624', '20240625', '20240626'] # 취득이 필요한 날짜 리스트
# 오늘날짜부터 1주일전까지만 가능함. c_date 시작일자는 20200101 부터 있음.
# 
# c_date = ['20240626', '20240620'] 
# c_date = ['20240619'] 
for i, (code, company_name) in enumerate(code_dic.items()):
    for collect_date in c_date:
        df_collect = get_time_price(url_base, code, collect_date)
        # add logic to save date for each date for each company
        f_name = f'{naver_dir}/{company_name[1]}_{collect_date}.csv'
        df_collect.to_csv(f_name)
        df_collect.to_pickle(f_name.replace('csv','pkl'))

## 일별시세

In [28]:
# 일자별 주식 데이터를 페이지별로 10개씩 취득
def get_piece_date_price(url_d):
    res = scraper.get(url_d, headers=headers)
    class_name = 'type2'
    df = pd.read_html(io.StringIO(str(res.text)), attrs={"class": class_name}, flavor=["lxml", "bs4"])[0]
    
    df = df.dropna(axis=0) # delete nan rows

    df.columns = ['date', 'close', 'close_change', 'open', 'high', 'low', 'volume'] # rename column
    df['date'] = df['date'].apply(lambda x: datetime.datetime.strptime(x, '%Y.%m.%d')) # convert character to datetime object
    
    # convert character values to integer value : 보합= 0, 하락= -, 상승= +
    df['close_change'] = df['close_change'].apply(lambda x: int(x[2:]) if x[:2] == '보합' 
                                  else (-int(x[4:].replace(',','')) if x[:2] == '하락' 
                                        else int(x[4:].replace(',',''))))  # convert characters to int
    # define variable types
    df['open'] = df['open'].astype(int)
    df['high'] = df['high'].astype(int)
    df['low'] = df['low'].astype(int)
    df['close'] = df['close'].astype(int)
    df['volume'] = df['volume'].astype(int)
    
    df = df[['date', 'open', 'high', 'low', 'close', 'close_change', 'volume']]  # rearrange columns

    return df

In [29]:
# 10개씩의 일자별 데이터를 원하는 일자부터 현재일자까지 합하어 취득
def get_date_price(url_base_d, code_com):
    
    page_num = 1
    
    # make first data frame
    page = str(page_num)
    url_date = url_base_d + '?code=' + code_com + '&page=' + page
    df_base = get_piece_date_price(url_date)
    
    page_num = page_num + 1
    
    while True:
        page = str(page_num)
        
        url_date = url_base_d + '?code=' + code_com + '&page=' + page
        df_p = get_piece_date_price(url_date)
    
        df_base = concat_df(df_base, df_p, 'date', 0)  # df concat후 'time' column을 기준으로 중복제거 후 0 column을 기준으로 정렬시킴.

        startdate_str = '2020/1/2 00:00:00' # 데이터 수집 시작 일자, startdate_str
        startdate = datetime.datetime.strptime(startdate_str, '%Y/%m/%d %H:%M:%S')

        if (startdate == df_base['date']).any(): # 시작일자와 일치하는 row가 있으면 더 이상 진행하지 않음.
            break

        page_num = page_num + 1

    return df_base

In [31]:
# naver_dir = 'data/naver_finance/date_data'
naver_dir = 'date_data'

url_base = 'https://finance.naver.com/item/sise_day.naver'  # sise_day

code_dic = {'005930': ['삼성전자', 'sec'], '005380': ['현대차', 'hyunmotor'], 
        '035420': ['NAVER', 'naver'], '033780': ['KT&G', 'ktng']}

# code_dic = {'005930': ['삼성전자', 'sec'],}

for i, (code, company_name) in enumerate(code_dic.items()):
    df_collect = get_date_price(url_base, code)
    f_name = f'{naver_dir}/{company_name[1]}.csv'
    df_collect.to_csv(f_name)
    df_collect.to_pickle(f_name.replace('csv','pkl'))

In [32]:
df_collect.tail(10)

,date,open,high,low,close,close_change,volume
1095,2024-06-05,85500,86100,85000,85500,300,178185
1096,2024-06-07,86300,86900,86000,86700,1200,285464
1097,2024-06-10,85900,86700,85500,85900,-800,159538
1098,2024-06-11,86000,86900,85900,86500,600,185079
1099,2024-06-12,86100,86800,86000,86500,0,126821
1100,2024-06-13,86500,87300,85900,85900,-600,349698
1101,2024-06-14,85300,86800,85300,86800,900,490156
1102,2024-06-17,86500,86800,86000,86800,0,104525
1103,2024-06-18,86700,87000,86100,86500,-300,213667
1104,2024-06-19,87200,87300,86000,86400,-100,153855


## 여기까지